# AN2DL [2025–2026] — Time Series Classification (Stratified K-Fold, SMOTE, XGBoost)

**NOTEBOOK BY thenegatives**


**Burchini - Collovigh - Corti - Ravasio**

## Google Drive (opzionale)

In [24]:
import sys, os

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount("/gdrive")
    current_dir = "/gdrive/My Drive/AN2DL//"
    try:
        os.chdir(current_dir)
    except FileNotFoundError:
        print("⚠️ Aggiorna `current_dir` alla tua cartella dati.")
else:
    current_dir = "."
print("Working dir:", os.getcwd())


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
Working dir: /gdrive/My Drive/AN2DL


## Librerie & seed

In [25]:
import os, random, warnings
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
from torch.optim import AdamW
from torch.cuda.amp import GradScaler, autocast

from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, classification_report

import matplotlib.pyplot as plt

SEED = 42
warnings.simplefilter("ignore")

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


## Configurazione

In [26]:
class CFG:
    n_folds = 10
    epochs = 65
    batch_size = 128
    lr = 5e-4
    weight_decay = 1e-4
    max_grad_norm = 2.0
    patience = 10
    label_smoothing = 0.01
    use_weighted_sampler = False

    hidden1 = 256
    hidden2 = 128
    dropout = 0.20

    train_csv = "pirate_pain_train.csv"
    train_labels_csv = "pirate_pain_train_labels.csv"
    test_csv = "pirate_pain_test.csv"

cfg = CFG()


## Caricamento dati

In [27]:
train_df = pd.read_csv(cfg.train_csv)
labels_df = pd.read_csv(cfg.train_labels_csv)
test_df  = pd.read_csv(cfg.test_csv)

data = pd.merge(train_df, labels_df, on="sample_index", how="left")

print("Train (time-step):", data.shape, "| Test (time-step):", test_df.shape)
print("Colonne:", list(data.columns)[:10], "...")


Train (time-step): (105760, 41) | Test (time-step): (211840, 40)
Colonne: ['sample_index', 'time', 'pain_survey_1', 'pain_survey_2', 'pain_survey_3', 'pain_survey_4', 'n_legs', 'n_hands', 'n_eyes', 'joint_00'] ...


## Selezione feature (35 input)

In [28]:
drop_cols = ["time", "n_legs", "n_hands", "n_eyes"]
feature_cols = [c for c in data.columns if c not in (["sample_index","label"] + drop_cols)]
assert len(feature_cols) == 35, f"Attese 35 feature, trovate {len(feature_cols)}"

print("Numero feature:", len(feature_cols))
print("Esempio:", feature_cols[:8])


Numero feature: 35
Esempio: ['pain_survey_1', 'pain_survey_2', 'pain_survey_3', 'pain_survey_4', 'joint_00', 'joint_01', 'joint_02', 'joint_03']


## Encoding etichette e tabella sequenze

In [29]:
label_encoder = {"no_pain":0, "low_pain":1, "high_pain":2}
inv_label_encoder = {v:k for k,v in label_encoder.items()}
data["label_encoded"] = data["label"].map(label_encoder).astype(int)

seq_df = data[["sample_index","label_encoded"]].drop_duplicates().reset_index(drop=True)
print("Soggetti:", len(seq_df))
print(seq_df["label_encoded"].value_counts().sort_index())


Soggetti: 661
label_encoded
0    511
1     94
2     56
Name: count, dtype: int64


## Helper: scaler, dataset, modello (MLP)

In [30]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
from torch.optim import AdamW
from torch.cuda.amp import GradScaler, autocast

def fit_minmax(X):
    mn = X.min(axis=0)
    mx = X.max(axis=0)
    denom = mx - mn
    denom = np.where(denom == 0.0, 1.0, denom)
    return mn, denom

def apply_minmax(X, mn, denom):
    return (X - mn) / denom

class NeuralNetwork(nn.Module):
    def __init__(self, in_dim=35, h1=128, h2=64, out_dim=3, dropout=0.10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, h1),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(h1, h2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(h2, out_dim)
        )
    def forward(self, x):
        return self.net(x)

def make_loader(X, y=None, batch_size=256, shuffle=False, sampler=None):
    if y is None:
        ds = TensorDataset(torch.from_numpy(X.astype(np.float32)))
    else:
        ds = TensorDataset(torch.from_numpy(X.astype(np.float32)),
                           torch.from_numpy(y.astype(np.int64)))
    return DataLoader(ds, batch_size=batch_size, shuffle=(sampler is None and shuffle),
                      sampler=sampler, num_workers=2, pin_memory=True)


## Train & Eval

In [ ]:
from sklearn.metrics import f1_score, classification_report
import torch

def train_one_epoch(model, loader, optimizer, criterion, scaler=None, max_grad_norm=2.0):
    model.train()
    total_loss, total_correct, total_samples = 0.0, 0, 0
    for x, y in loader:
        x = x.to(device); y = y.to(device)
        optimizer.zero_grad(set_to_none=True)
        with autocast(enabled=(scaler is not None)):
            logits = model(x)
            loss = criterion(logits, y)
        if scaler is not None:
            scaler.scale(loss).backward()
            if max_grad_norm is not None:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            scaler.step(optimizer); scaler.update()
        else:
            loss.backward()
            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
        total_loss += loss.item() * x.size(0)
        total_correct += (logits.argmax(1) == y).sum().item()
        total_samples += x.size(0)
    return total_loss/max(1,total_samples), total_correct/max(1,total_samples)

@torch.no_grad()
def evaluate(model, loader, criterion=None):
    model.eval()
    total_loss, total_correct, total_samples = 0.0, 0, 0
    all_preds, all_tgts = [], []
    for x, y in loader:
        x = x.to(device); y = y.to(device)
        logits = model(x)
        if criterion is not None:
            loss = criterion(logits, y)
            total_loss += loss.item() * x.size(0)
        total_correct += (logits.argmax(1) == y).sum().item()
        total_samples += x.size(0)
        all_preds.append(logits.argmax(1).cpu().numpy())
        all_tgts.append(y.cpu().numpy())
    avg_loss = total_loss/max(1,total_samples) if total_samples>0 else None
    acc = total_correct/max(1,total_samples) if total_samples>0 else None
    return avg_loss, acc, np.concatenate(all_preds), np.concatenate(all_tgts)


def _get_x_from_batch(batch):
    # needed as batch may be a tensor, (x,), (x,y) or a dict
    if isinstance(batch, (list, tuple)):
        x = batch[0]
    elif isinstance(batch, dict):
        # try common keys
        for k in ('x', 'inputs', 'features'):
            if k in batch:
                x = batch[k]
                break
        else:
            raise ValueError("Impossibile estrarre le feature dal batch di tipo dict.")
    else:
        x = batch
    return x
@torch.no_grad()
def predict_proba(model, loader):
    model.eval()
    all_probs = []
    with torch.no_grad():
        for batch in loader:
            x = _get_x_from_batch(batch)
            x = x.to(device, non_blocking=True)  # uses previously defined variable "device"
            logits = model(x)
            probs = torch.softmax(logits, dim=1)
            all_probs.append(probs.detach().cpu().numpy())
    return np.concatenate(all_probs, axis=0)


## Stratified K-Fold (group-by `sample_index`) + OOF + XGBoost

In [36]:
X_all = data[feature_cols].values
y_all = data["label_encoded"].values
sid_all = data["sample_index"].values
X_test_all = test_df[feature_cols].values
sid_test_all = test_df["sample_index"].values

unique_sids = seq_df["sample_index"].values
sid_to_label = dict(zip(seq_df["sample_index"].values, seq_df["label_encoded"].values))
y_seq = np.array([sid_to_label[s] for s in unique_sids])

print("K-Fold data initialization complete.")

K-Fold data initialization complete.


In [ ]:
from xgboost import XGBClassifier
import xgboost as xgb

# Ensemble weights
ENSEMBLE_WEIGHT_NN = 0.5
ENSEMBLE_WEIGHT_XGB = 0.5

def train_xgb(X_tr, y_tr, X_va, y_va, random_state=SEED):
    model_xgb = XGBClassifier(
        objective='multi:softprob',  # For multi-class classification with probabilities
        num_class=3,                 # Number of classes
        eval_metric='mlogloss',      # LogLoss for multi-class
        use_label_encoder=False,     # Suppress warning
        random_state=random_state,
        n_estimators=500,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.7,
        colsample_bytree=0.7,
        gamma=0.1,
        tree_method='hist', # Faster for larger datasets
        n_jobs=-1
    )

    model_xgb.fit(X_tr, y_tr,
                  eval_set=[(X_va, y_va)],
                  verbose=False)
    return model_xgb

# Initialize OOF and test probability arrays for XGBoost
oof_probs_xgb = np.zeros((len(X_all), 3), dtype=np.float32)
test_fold_probs_xgb = []

print("XGBoost setup complete and ensemble weights defined.")

XGBoost setup complete and ensemble weights defined.


In [ ]:
oof_probs_nn = np.zeros((len(X_all), 3), dtype=np.float32)
oof_preds_nn = np.zeros(len(X_all), dtype=np.int64)
oof_true   = y_all.copy()
test_fold_probs_nn = []

models_xgb = [] # To store trained XGBoost models

from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE

skf = StratifiedKFold(n_splits=cfg.n_folds, shuffle=True, random_state=SEED)
for fold, (tr_seq_idx, va_seq_idx) in enumerate(skf.split(unique_sids, y_seq), start=1):
    tr_sids = unique_sids[tr_seq_idx]; va_sids = unique_sids[va_seq_idx]
    tr_mask = np.isin(sid_all, tr_sids); va_mask = np.isin(sid_all, va_sids)
    X_tr, y_tr = X_all[tr_mask], y_all[tr_mask]
    X_va, y_va = X_all[va_mask], y_all[va_mask]

    # Minority class oversampling is performed with SMOTE
    class_counts = np.bincount(y_tr)
    if len(class_counts) > 2:
        majority_cls = int(np.argmax(class_counts))
        minority_cls = 2  # high_pain
        if class_counts[minority_cls] < class_counts[majority_cls]:
            smote = SMOTE(sampling_strategy={minority_cls: int(class_counts[majority_cls])}, random_state=SEED)
            X_tr, y_tr = smote.fit_resample(X_tr, y_tr)

    mn, denom = fit_minmax(X_tr)
    X_tr_s = apply_minmax(X_tr, mn, denom)
    X_va_s = apply_minmax(X_va, mn, denom)
    X_te_s = apply_minmax(X_test_all, mn, denom)

    # Train and predict with XGBoost
    print(f"\n===== FOLD {fold}/{cfg.n_folds} \u2014 XGBoost Training ====")
    xgb_model = train_xgb(X_tr_s, y_tr, X_va_s, y_va, random_state=SEED + fold) # Use fold in seed for variation
    models_xgb.append(xgb_model)

    # XGBoost OOF and test predictions
    oof_probs_xgb[va_mask] = xgb_model.predict_proba(X_va_s)
    test_fold_probs_xgb.append(xgb_model.predict_proba(X_te_s))
    print(f"XGBoost F1 micro on validation: {f1_score(y_va, xgb_model.predict(X_va_s), average='micro'):.4f}")

    # NN Training and Prediction
    sampler = None
    if cfg.use_weighted_sampler:
        class_sample_count = np.array([np.sum(y_tr == t) for t in np.unique(y_tr)])
        weight_per_class = 1.0 / np.maximum(class_sample_count, 1)
        weights = np.array([weight_per_class[t] for t in y_tr])
        sampler = WeightedRandomSampler(weights=weights, num_samples=len(weights), replacement=True)

    classes = np.unique(y_tr)
    class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_tr)
    full_w = np.ones(3, dtype=np.float32)
    for i, c in enumerate(classes):
        full_w[c] = class_weights[i]
    class_weights_t = torch.tensor(full_w, dtype=torch.float32, device=device)

    criterion = nn.CrossEntropyLoss(weight=class_weights_t, label_smoothing=cfg.label_smoothing)

    tr_loader = make_loader(X_tr_s, y_tr, batch_size=cfg.batch_size, shuffle=(sampler is None), sampler=sampler)
    va_loader = make_loader(X_va_s, y_va, batch_size=cfg.batch_size, shuffle=False)
    te_loader = make_loader(X_te_s, batch_size=cfg.batch_size, shuffle=False)

    model = NeuralNetwork(in_dim=len(feature_cols), h1=cfg.hidden1, h2=cfg.hidden2, out_dim=3, dropout=cfg.dropout).to(device)
    optimizer = AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    scaler = GradScaler()

    best_f1, best_state, no_improve = -1.0, None, 0
    print(f"\n===== FOLD {fold}/{cfg.n_folds} \u2014 NN Training \u2014 train rows: {len(X_tr)}, val rows: {len(X_va)} ====")
    for epoch in range(1, cfg.epochs+1):
        loss_tr, acc_tr = train_one_epoch(model, tr_loader, optimizer, criterion, scaler=scaler, max_grad_norm=cfg.max_grad_norm)
        loss_va, acc_va, preds_va, tgts_va = evaluate(model, va_loader, criterion=criterion)
        from sklearn.metrics import f1_score
        f1_va = f1_score(tgts_va, preds_va, average="micro")
        print(f"Epoch {epoch:02d} | loss_tr {loss_tr:.4f} acc_tr {acc_tr:.4f} | loss_va {loss_va:.4f} acc_va {acc_va:.4f} | f1_va {f1_va:.4f}")
        if f1_va > best_f1:
            best_f1 = f1_va
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= cfg.patience:
                print(f"Early stopping \u2014 best F1: {best_f1:.4f}")
                break

    if best_state is not None:
        model.load_state_dict({k: v.to(device) for k, v in best_state.items()})

    _, _, preds_va, tgts_va = evaluate(model, va_loader, criterion=None)
    oof_preds_nn[va_mask] = preds_va
    va_probs_nn = predict_proba(model, va_loader)
    oof_probs_nn[va_mask] = va_probs_nn

    from sklearn.metrics import classification_report
    rep = classification_report(tgts_va, preds_va, target_names=["no_pain","low_pain","high_pain"], digits=4)
    print("Fold NN report:\n", rep)

    test_probs_nn = predict_proba(model, te_loader)
    test_fold_probs_nn.append(test_probs_nn)

# Calculate OOF F1 for NN
oof_f1_micro_nn = f1_score(oof_true, oof_preds_nn, average="micro")
print(f"\nNN OOF F1 micro: {oof_f1_micro_nn:.4f}")
print(classification_report(oof_true, oof_preds_nn, target_names=["no_pain","low_pain","high_pain"], digits=4))

# Calculate OOF F1 for XGBoost
oof_preds_xgb = np.argmax(oof_probs_xgb, axis=1)
oof_f1_micro_xgb = f1_score(oof_true, oof_preds_xgb, average="micro")
print(f"\nXGBoost OOF F1 micro: {oof_f1_micro_xgb:.4f}")
print(classification_report(oof_true, oof_preds_xgb, target_names=["no_pain","low_pain","high_pain"], digits=4))


===== FOLD 1/10 — XGBoost Training ====
XGBoost F1 micro on validation: 0.9198

===== FOLD 1/10 — NN Training — train rows: 160480, val rows: 10720 ====
Epoch 01 | loss_tr 0.7233 acc_tr 0.7016 | loss_va 0.7505 acc_va 0.7398 | f1_va 0.7398
Epoch 02 | loss_tr 0.4610 acc_tr 0.8407 | loss_va 0.6090 acc_va 0.7938 | f1_va 0.7938
Epoch 03 | loss_tr 0.3644 acc_tr 0.8843 | loss_va 0.4920 acc_va 0.8609 | f1_va 0.8609
Epoch 04 | loss_tr 0.3112 acc_tr 0.9088 | loss_va 0.4960 acc_va 0.8590 | f1_va 0.8590
Epoch 05 | loss_tr 0.2777 acc_tr 0.9232 | loss_va 0.5221 acc_va 0.8350 | f1_va 0.8350
Epoch 06 | loss_tr 0.2531 acc_tr 0.9336 | loss_va 0.4546 acc_va 0.8744 | f1_va 0.8744
Epoch 07 | loss_tr 0.2348 acc_tr 0.9410 | loss_va 0.4899 acc_va 0.8599 | f1_va 0.8599
Epoch 08 | loss_tr 0.2208 acc_tr 0.9473 | loss_va 0.4321 acc_va 0.8992 | f1_va 0.8992
Epoch 09 | loss_tr 0.2101 acc_tr 0.9516 | loss_va 0.4359 acc_va 0.8777 | f1_va 0.8777
Epoch 10 | loss_tr 0.2012 acc_tr 0.9549 | loss_va 0.4044 acc_va 0.9032 |

In [22]:
test_mean_probs_nn = np.mean(np.stack(test_fold_probs_nn, axis=0), axis=0)
test_mean_probs_xgb = np.mean(np.stack(test_fold_probs_xgb, axis=0), axis=0)

# Ensemble OOF probabilities
ensembled_oof_probs = (oof_probs_nn * ENSEMBLE_WEIGHT_NN) + (oof_probs_xgb * ENSEMBLE_WEIGHT_XGB)
ensembled_oof_preds = np.argmax(ensembled_oof_probs, axis=1)

# Calculate ensembled OOF F1 micro score
oof_f1_micro_ensembled = f1_score(oof_true, ensembled_oof_preds, average="micro")
print(f"\nEnsembled OOF F1 micro: {oof_f1_micro_ensembled:.4f}")
print(classification_report(oof_true, ensembled_oof_preds, target_names=["no_pain","low_pain","high_pain"], digits=4))

# Ensemble test probabilities
ensembled_test_probs = (test_mean_probs_nn * ENSEMBLE_WEIGHT_NN) + (test_mean_probs_xgb * ENSEMBLE_WEIGHT_XGB)

df_test_pred = pd.DataFrame({
    "sample_index": sid_test_all,
    "p0": ensembled_test_probs[:,0],
    "p1": ensembled_test_probs[:,1],
    "p2": ensembled_test_probs[:,2],
})
agg = df_test_pred.groupby("sample_index")[["p0","p1","p2"]].mean().reset_index()
agg["pred_label_id"] = agg[["p0","p1","p2"]].values.argmax(axis=1)
inv_label_encoder = {0:"no_pain", 1:"low_pain", 2:"high_pain"}
agg["label"] = agg["pred_label_id"].map(inv_label_encoder)
print(agg["label"].value_counts())
agg.head()


Ensembled OOF F1 micro: 0.9359
              precision    recall  f1-score   support

     no_pain     0.9632    0.9697    0.9665     81760
    low_pain     0.8886    0.8668    0.8775     15040
   high_pain     0.7593    0.7438    0.7514      8960

    accuracy                         0.9359    105760
   macro avg     0.8704    0.8601    0.8651    105760
weighted avg     0.9353    0.9359    0.9356    105760

label
no_pain      1041
low_pain      180
high_pain     103
Name: count, dtype: int64


,sample_index,p0,p1,p2,pred_label_id,label
0,0,0.853606,0.051745,0.094649,0,no_pain
1,1,0.940932,0.016505,0.042564,0,no_pain
2,2,0.952717,0.015500,0.031783,0,no_pain
3,3,0.857329,0.028623,0.114048,0,no_pain
4,4,0.897993,0.011758,0.090249,0,no_pain


## Salvataggio `submission.csv`

In [ ]:
submission = agg[["sample_index","label"]].sort_values("sample_index").reset_index(drop=True)
submission.to_csv("submission.csv", index=False)
print("✔️ Salvato:", os.path.abspath("submission.csv"))
submission.head()